In [38]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import scipy.stats as stats
import os
import math
from tqdm import tqdm

In [39]:
''' функция для получения списка файлов из папки
'''
def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [40]:
''' предобработка изображения:
1. изменение цветовой системы с BGR на GRAY
2. размытие изображения
3. поиск контуров на изображении
4. изменение цветовой системы полученного изображения на двухцветную
5. удаление внутренних/внешних мелких деталий
'''
def preprocess(image, canny1, canny2):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (15,15), 0)
    img_canny = cv2.Canny(image, canny1, canny2)
    tresh, img_bin = cv2.threshold(img_canny, 127, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5,5),np.uint8)
    img_close = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel)
    img_open = cv2.morphologyEx(img_close, cv2.MORPH_OPEN, kernel)
    return img_open

In [41]:
''' нанесение найденных окружностей на изображение
'''
def show_circles(df_circles, image):
    for i in range(len(df_circles)):
        circle = df_circles.iloc[i]
        [x_center,y_center,radius] = circle[['x_center','y_center','radius']]
        cv2.circle(image, (int(x_center), int(y_center)), int(radius), (0, 255, 0), 2)
        cv2.circle(image,(int(x_center), int(y_center)),2,(0,0,255),3)
    return image

In [42]:
''' удаление окружностей по следующим принципам:
1. не входит во 2/3 квантиль по радиусу
2. выходит за пределы изображения
'''
def check_size(df_circles, sh):
    
    lower_bound = df_circles.radius.quantile(q=0.25)
    upper_bound = df_circles.radius.quantile(q=0.75)
    df_circles = df_circles[(df_circles.radius >= lower_bound) & (df_circles.radius <= upper_bound)]
    
    df_circles['max_x'] = df_circles['x_center'] + df_circles['radius']
    df_circles['min_x'] = df_circles['x_center'] - df_circles['radius']
    df_circles['max_y'] = df_circles['y_center'] + df_circles['radius']
    df_circles['min_y'] = df_circles['y_center'] - df_circles['radius']
     
    df_circles = df_circles[((df_circles['max_x']<=sh[1])*1+(df_circles['min_x']>=0)*1+(df_circles['max_y']<=sh[0])*1+(df_circles['min_y']>=0)*1) == 4]

    return df_circles[['x_center','y_center','radius']]

In [43]:
''' функция для определения характеристики IoU
'''
def bb_intersection_over_union(boxA, boxB):
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	iou = interArea / float(boxAArea + boxBArea - interArea)
	return iou

In [44]:
''' удаление окружностей, имеющих высокую оценку по IoU
'''
def non_max_supression(df_circles):
    df_circles['xA'] = 0
    df_circles['yA'] = 0
    df_circles['xB'] = 0
    df_circles['yB'] = 0

    for i in range(len(df_circles)):
        c = df_circles.iloc[i]
        df_circles['xA'].iloc[i] = c.x_center - c.radius
        df_circles['yA'].iloc[i] = c.y_center - c.radius
        df_circles['xB'].iloc[i] = c.x_center + c.radius
        df_circles['yB'].iloc[i] = c.y_center + c.radius

    list_circles = [0]
    for i in range(1,len(df_circles)):
        df_list_circles = df_circles.iloc[list_circles]
        counter=0
        for j in range(len(df_list_circles)):
            circle1=df_circles.iloc[i]
            circle2=df_circles.iloc[j]
            boxA=[circle1['xA'], circle1['yA'], circle1['xB'], circle1['yB']]
            boxB=[circle2['xA'], circle2['yA'], circle2['xB'], circle2['yB']]
            if bb_intersection_over_union(boxA, boxB)>0.1:
                counter=counter+1
        if counter==0:
            list_circles.append(i)

    df_circles=df_circles.iloc[list_circles]

    return df_circles

In [45]:
''' матчинг найденной окружности с заданными в описании
'''

def match(circle1,circle2,tresh):
    if np.abs(circle1.x_center-circle2.x_center)<tresh and np.abs(circle1.y_center-circle2.y_center)<tresh and np.abs(circle1.radius-circle2.radius)<tresh:
        result = 1
    else:
        result =0
    return result

In [46]:
''' получение метрик по предсказанию:
1. кол-во true positive
2. кол-во false negative
1. кол-во false positive
'''

def get_metrics(file,df_true,result_circles):
    trues = df_true[df_true.pic==file]
    preds = result_circles[result_circles.pic==file]
    trues['match'] = 0
    for i in range(len(trues)):
        for j in range(len(preds)):
            circle1 = trues.iloc[i]
            circle2 = preds.iloc[j]
            if match(circle1,circle2,20)==1:
                trues['match'].iloc[i]=1
    tp = len(trues[trues['match']==1])
    fn = len(trues[trues['match']==0])
    fp = len(preds) - len(trues[trues['match']==1])
    return tp, fn, fp

In [47]:
''' полный процесс получения изображений с предсказанными окружностями и файла с оценкой качества предсказаний:
1. чтение изображения из фала
2. предобработка файла
3. поиск окружностей на изображении
4. проверка размера окружностей
5. удаление пересекающихся окружностей
6. отрисовка окружностей
7. запись изображений с предсказанными окружностями
8. подсчет метрик
'''
def full_process(src_path, descr_path, canny1, canny2, hough1, hough2):
    path = src_path
    data = {'x_center': [], 'y_center': [], 'radius': [], 'pic': []}
    result_circles = pd.DataFrame(data)

    for file in files(path):
        image_path = f'{path}/{file}'
        result_path = f'{path}/res'
        image = cv2.imread(image_path)
        sh = image.shape
        img_preprocessed = preprocess(image, canny1=canny1, canny2=canny2)
        circles = cv2.HoughCircles(img_preprocessed, cv2.HOUGH_GRADIENT,1, 50, param1=hough1, param2=hough2, minRadius=50, maxRadius=500)

        try:
            df_circles = pd.DataFrame(circles[0,:].T[:3].T).rename(columns={0: "x_center", 1: "y_center", 2: "radius"})
            df_circles = check_size(df_circles, sh) 
            df_circles = non_max_supression(df_circles)
            
            df_circles['pic'] = str(file)
            result_circles = pd.concat([result_circles,df_circles])   
            img_circle = show_circles(df_circles, image)
            os.chdir(result_path)
            cv2.imwrite(file, img_circle)
        except:
            print(file)

    df_true = pd.read_csv(descr_path, sep=",", names=['pic','x_center','y_center','radius'])

    data = {'pic': [], 'tp': [], 'fn': [], 'fp': []}
    result =  pd.DataFrame(data)
                            
    for file in files(path):
        tp, fn, fp = get_metrics(file, df_true, result_circles)
        data = {'pic': [file], 'tp': [tp], 'fn': [fn], 'fp': [fp]}
        file_result =  pd.DataFrame(data)
        result = pd.concat([result,file_result])
        result['recall'] = result['tp']/( result['tp']+ result['fn'])
        result['precision'] = result['tp']/( result['tp']+ result['fp'])
        result['f1'] = 2*result['recall']*result['precision']/(result['recall']+result['precision'])
        result = result[(result['tp']+result['fn']+result['fp'])>0]

    return result

    

In [ ]:
path = "C:/find_coins/coins"
descr_path = f'C:/find_coins/circles_description.txt'
result = full_process(path, descr_path, 10, 120, 300, 7)
f1_mean = np.mean(result.sort_values(by='f1',ascending=False)['f1'])

In [50]:
''' вывод метрики f1 по 10ти изображениям
'''
print('Total average F1 score:', f1_mean)

Total average F1 score: 0.8411111111111111


In [ ]:
''' 
# grid search для поиска оптимальных параметров функции cv2.Canny и cv2.HoughCircle

canny1_list = []
canny2_list = []
hough1_list = []
hough2_list = []
f1_list = []

for canny1 in range(10,30,10):
    for canny2 in range(80,130,20):
        for hough1 in range(250,350,50):
            for hough2 in range(1,25,3):
                f1 = full_process(path, descr_path, canny1, canny2, hough1, hough2)
                canny1_list.append(canny1)
                canny2_list.append(canny2)
                hough1_list.append(hough1)
                hough2_list.append(hough2)
                f1_list.append(f1)

data = {'canny1': canny1_list, 'canny2': canny2_list, 'hough1': hough1_list, 'hough2': hough2_list, 'hough2': hough2_list,'f1': f1_list}
df = pd.DataFrame(data)

'''